In [4]:
import pandas as pd
import numpy as np
import sys
import os

# Allows to get the module in utils
sys.path.append(os.path.abspath(".."))


from utils.load import load_data
from utils.preprocessing import preprocess
from utils.split import split_data

from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.compose import ColumnTransformer

import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import optuna

(4507, 20) (1409, 20) (1127, 20) (1409, 20) (1127,) (1409,)


2025-05-22 14:03:26.158479: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 14:03:26.412291: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 14:03:26.631759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747915406.851481   13538 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747915406.913759   13538 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747915407.335311   13538 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [5]:
# Load data
df = load_data()

# Split and preprocess datas set
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df)
X_train_processed, X_test_processed, X_val_processed, y_test_encoding, y_train_encoded, y_val_encoding, pipeline, le = preprocess(X_train, X_val, X_test, y_train, y_val, y_test)

In [6]:
X_train_dense = X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed
X_val_dense = X_val_processed.toarray() if hasattr(X_val_processed, "toarray") else X_val_processed
X_test_dense = X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed

In [7]:
y_train_vector = y_train_encoded.reshape(-1)
y_val_vector = y_val_encoding.reshape(-1)
y_test_vector = y_test_encoding.reshape(-1)

In [8]:
input_dim = X_train_dense.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy', 'recall', 'auc']
)

model.summary()

E0000 00:00:1747915433.460318   13538 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747915433.461279   13538 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,209 (24.25 KB)

 Trainable params: 6,209 (24.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
checkpoint_path = "checkpoints/churn_models.keras"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',  
    save_best_only=True,# sauvegarde le modèle qui maximise l’accuracy de validation
    verbose=1
)

In [ ]:
history = model.fit(
    X_train_dense, y_train_vector,
    validation_data=(X_val_dense, y_val_vector),
    epochs=20,
    batch_size=16,
    callbacks=[model_ckpt],
    verbose=1
)

In [5]:
from sklearn.metrics import roc_auc_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def objective(trial):
    # Hyperparamètres à tester
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 50)
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_units = trial.suggest_int('n_units', 16, 128, step=16)

    # Définir le modèle
    model = Sequential()
    model.add(Input(shape=(X_train_dense.shape[1],)))
    for _ in range(n_layers):
        model.add(Dense(n_units, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['AUC']
    )

    # Early stopping pour éviter l’overfitting
    early_stop = EarlyStopping(monitor='val_auc', mode='max', patience=5, restore_best_weights=True)

    # Entraînement
    model.fit(
        X_train_dense, y_train_vector,
        validation_data=(X_val_dense, y_val_vector),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop]
    )

    # Prédictions sur les probas
    y_pred_proba = model.predict(X_val_dense)
    auc = roc_auc_score(y_val_vector, y_pred_proba)

    return auc

In [7]:
study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name='first_test')
study.optimize(objective, n_trials=20)


[I 2025-05-22 11:10:09,385] A new study created in RDB with name: first_test
/tmp/ipykernel_150382/1618310837.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


[I 2025-05-22 11:10:28,745] Trial 0 finished with value: 0.8432254051346679 and parameters: {'learning_rate': 0.028454209218404886, 'batch_size': 32, 'epochs': 26, 'n_layers': 3, 'n_units': 16}. Best is trial 0 with value: 0.8432254051346679.
/tmp/ipykernel_150382/1618310837.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-22 11:10:43,947] Trial 1 finished with value: 0.847252516439662 and parameters: {'learning_rate': 0.00018303419708245325, 'batch_size': 64, 'epochs': 39, 'n_layers': 1, 'n_units': 64}. Best is trial 1 with value: 0.847252516439662.
/tmp/ipykernel_150382/1618310837.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


...

[I 2025-05-22 11:15:08,431] Trial 17 finished with value: 0.8409593976701726 and parameters: {'learning_rate': 0.0006162567504444866, 'batch_size': 64, 'epochs': 18, 'n_layers': 3, 'n_units': 96}. Best is trial 1 with value: 0.847252516439662.
/tmp/ipykernel_150382/1618310837.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


[I 2025-05-22 11:15:18,848] Trial 18 finished with value: 0.8428578352963987 and parameters: {'learning_rate': 0.0015600238169410593, 'batch_size': 64, 'epochs': 32, 'n_layers': 1, 'n_units': 112}. Best is trial 1 with value: 0.847252516439662.
/tmp/ipykernel_150382/1618310837.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


[I 2025-05-22 11:15:26,704] Trial 19 finished with value: 0.8396587659347583 and parameters: {'learning_rate': 0.00010416545711390481, 'batch_size': 64, 'epochs': 24, 'n_layers': 2, 'n_units': 48}. Best is trial 1 with value: 0.847252516439662.


In [8]:
print("Best trial:")
trial = study.best_trial
print(f"AUC: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")


Best trial:
AUC: 0.847252516439662
Params:
  learning_rate: 0.00018303419708245325
  batch_size: 64
  epochs: 39
  n_layers: 1
  n_units: 64


In [15]:
# Test with class_weights
from sklearn.utils import class_weight

# 
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_vector),
    y=y_train_vector
)
class_weights = dict(enumerate(class_weights))


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 50)
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_units = trial.suggest_int('n_units', 32, 128, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)

    model = tf.keras.Sequential()
    model.add(Input(shape=(X_train_dense.shape[1],)))
    for _ in range(n_layers):
        model.add(Dense(n_units, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['AUC']
    )

    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=5, restore_best_weights=True)

    model.fit(
        X_train_dense, y_train_vector,
        validation_data=(X_val_dense, y_val_vector),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop],
        class_weight=class_weights  
    )

    y_pred_proba = model.predict(X_val_dense)
    return roc_auc_score(y_val_vector, y_pred_proba)


In [12]:
study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name='Class_weight_and_dropout2')
study.optimize(objective, n_trials=30)



[I 2025-05-22 12:11:03,833] A new study created in RDB with name: Class_weight_and_dropout2


/tmp/ipykernel_150382/3181749786.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


[I 2025-05-22 12:12:02,446] Trial 0 finished with value: 0.8113195353270969 and parameters: {'learning_rate': 0.007214512784914, 'batch_size': 16, 'epochs': 50, 'n_layers': 1, 'n_units': 96, 'dropout_rate': 0.08322172010964218}. Best is trial 0 with value: 0.8113195353270969.
/tmp/ipykernel_150382/3181749786.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


...

[I 2025-05-22 12:23:34,110] Trial 27 finished with value: 0.8393639022183446 and parameters: {'learning_rate': 0.00027615710791436263, 'batch_size': 16, 'epochs': 36, 'n_layers': 2, 'n_units': 96, 'dropout_rate': 0.0380263059522773}. Best is trial 11 with value: 0.851675472185869.
/tmp/ipykernel_150382/3181749786.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


[I 2025-05-22 12:23:57,242] Trial 28 finished with value: 0.8500880551920249 and parameters: {'learning_rate': 0.00012683793172531718, 'batch_size': 16, 'epochs': 27, 'n_layers': 1, 'n_units': 32, 'dropout_rate': 0.13344333046428264}. Best is trial 11 with value: 0.851675472185869.
/tmp/ipykernel_150382/3181749786.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


[I 2025-05-22 12:24:09,261] Trial 29 finished with value: 0.8157990402791915 and parameters: {'learning_rate': 1.8322249231580193e-05, 'batch_size': 32, 'epochs': 27, 'n_layers': 1, 'n_units': 32, 'dropout_rate': 0.09189104678954456}. Best is trial 11 with value: 0.851675472185869.


In [13]:
print("Best trial:")
trial = study.best_trial
print(f"AUC: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")


Best trial:
AUC: 0.851675472185869
Params:
  learning_rate: 2.7034820560657534e-05
  batch_size: 16
  epochs: 34
  n_layers: 2
  n_units: 128
  dropout_rate: 0.01560633556745207


In [12]:
from sklearn.metrics import roc_auc_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 50)
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_units = trial.suggest_int('n_units', 32, 128, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])

    optimizer = {
        'adam': Adam(learning_rate=learning_rate),
        'rmsprop': tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    }[optimizer_name]
    model = tf.keras.Sequential()
    model.add(Input(shape=(X_train_dense.shape[1],)))
    for _ in range(n_layers):
        model.add(Dense(n_units, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout_rate))
        model.add(Dense(n_units, activation=activation))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['AUC']
    )

    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=5, restore_best_weights=True)

    model.fit(
        X_train_dense, y_train_vector,
        validation_data=(X_val_dense, y_val_vector),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop],
        class_weight=class_weights  
    )

    y_pred_proba = model.predict(X_val_dense)
    return roc_auc_score(y_val_vector, y_pred_proba)


In [16]:
study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name='Different_activators_optimizers_2')
study.optimize(objective, n_trials=30)

[I 2025-05-22 14:05:58,973] A new study created in RDB with name: Different_activators_optimizers_2


/tmp/ipykernel_13538/155750586.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


[I 2025-05-22 14:06:13,950] Trial 0 finished with value: 0.8473131048745417 and parameters: {'learning_rate': 0.0003370233345318563, 'batch_size': 32, 'epochs': 18, 'n_layers': 1, 'n_units': 64, 'dropout_rate': 0.022031014357149714, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 0 with value: 0.8473131048745417.
/tmp/ipykernel_13538/155750586.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


...

[I 2025-05-22 14:16:58,485] Trial 27 finished with value: 0.8458589824374323 and parameters: {'learning_rate': 0.0006587910462592623, 'batch_size': 32, 'epochs': 49, 'n_layers': 1, 'n_units': 96, 'dropout_rate': 0.4767644889479124, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 22 with value: 0.8498739760554506.
/tmp/ipykernel_13538/155750586.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


[I 2025-05-22 14:17:11,696] Trial 28 finished with value: 0.8448087828995202 and parameters: {'learning_rate': 0.00013742868168372483, 'batch_size': 32, 'epochs': 14, 'n_layers': 2, 'n_units': 96, 'dropout_rate': 0.4061137419343165, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 22 with value: 0.8498739760554506.
/tmp/ipykernel_13538/155750586.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


[I 2025-05-22 14:17:20,348] Trial 29 finished with value: 0.8447239590906888 and parameters: {'learning_rate': 0.000384635429299998, 'batch_size': 64, 'epochs': 18, 'n_layers': 1, 'n_units': 128, 'dropout_rate': 0.3561023400735187, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 22 with value: 0.8498739760554506.


In [17]:
print("Best trial:")
trial = study.best_trial
print(f"AUC: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")


Best trial:
AUC: 0.8498739760554506
Params:
  learning_rate: 0.00017869599117172476
  batch_size: 32
  epochs: 16
  n_layers: 1
  n_units: 96
  dropout_rate: 0.49702772153539576
  activation: tanh
  optimizer: adam


In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
import numpy as np

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 50)
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_units = trial.suggest_int('n_units', 32, 128, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])

    # Optimizer mapping
    optimizer = {
        'adam': Adam(learning_rate=learning_rate),
        'rmsprop': RMSprop(learning_rate=learning_rate)
    }[optimizer_name]

    # Compute class weights
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_vector),
        y=y_train_vector
    )
    class_weights = dict(enumerate(class_weights))

    # Build model
    model = tf.keras.Sequential()
    model.add(Input(shape=(X_train_dense.shape[1],)))
    for _ in range(n_layers):
        model.add(Dense(n_units, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['AUC']
    )

    # Early stopping
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc', mode='max', patience=5,
        restore_best_weights=True, verbose=0
    )

    # Train model
    model.fit(
        X_train_dense, y_train_vector,
        validation_data=(X_val_dense, y_val_vector),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop],
        class_weight=class_weights
    )

    # Evaluate
    y_pred_proba = model.predict(X_val_dense)
    auc = roc_auc_score(y_val_vector, y_pred_proba)
    return auc


In [ ]:
study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name='test_all')
study.optimize(objective, n_trials=30)

[I 2025-05-22 16:13:49,737] A new study created in RDB with name: test_all
/tmp/ipykernel_13538/2735973175.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-05-22 16:14:25,541] Trial 0 finished with value: 0.8471636534018386 and parameters: {'learning_rate': 0.001499215413040741, 'batch_size': 32, 'epochs': 48, 'n_layers': 1, 'n_units': 96, 'dropout_rate': 0.09982637499373115, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 0 with value: 0.8471636534018386.
/tmp/ipykernel_13538/2735973175.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


[I 2025-05-22 16:14:39,649] Trial 1 finished with value: 0.8272664113873944 and parameters: {'learning_rate': 0.005470679625854893, 'batch_size': 64, 'epochs': 32, 'n_layers': 2, 'n_units': 32, 'dropout_rate': 0.10477560174173628, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 0 with value: 0.8471636534018386.
/tmp/ipykernel_13538/2735973175.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


...

[I 2025-05-22 16:26:43,629] Trial 27 finished with value: 0.8465698867400191 and parameters: {'learning_rate': 0.0007125596865928948, 'batch_size': 32, 'epochs': 18, 'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.3488311000393136, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 19 with value: 0.8494902493012134.
/tmp/ipykernel_13538/2735973175.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


[I 2025-05-22 16:27:09,492] Trial 28 finished with value: 0.8439645840401985 and parameters: {'learning_rate': 2.4185374605449687e-05, 'batch_size': 32, 'epochs': 33, 'n_layers': 2, 'n_units': 96, 'dropout_rate': 0.1925491176035854, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 19 with value: 0.8494902493012134.
/tmp/ipykernel_13538/2735973175.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/home/utilisateur/Documents/deeplearning_brief/DeepLearning-Project/.venv/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


[I 2025-05-22 16:27:29,330] Trial 29 finished with value: 0.8433627389203949 and parameters: {'learning_rate': 0.0002785122839008607, 'batch_size': 32, 'epochs': 27, 'n_layers': 3, 'n_units': 64, 'dropout_rate': 0.2453536777681571, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 19 with value: 0.8494902493012134.


In [20]:
print("Best trial:")
trial = study.best_trial
print(f"AUC: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"  {key}: {value}")


Best trial:
AUC: 0.8494902493012134
Params:
  learning_rate: 7.347434405768566e-05
  batch_size: 32
  epochs: 35
  n_layers: 2
  n_units: 96
  dropout_rate: 0.16149345969880197
  activation: relu
  optimizer: adam
